The following additional libraries are needed to run this
notebook. Note that running on Colab is experimental, please report a Github
issue if you have any problem.

In [ ]:
!pip install d2l==0.17.1


# Implementação Concisa de Regressão Linear
:label:`sec_linear_concise`


Amplo e intenso interesse em *deep learning* nos últimos anos
inspiraram empresas, acadêmicos e amadores
para desenvolver uma variedade de estruturas de código aberto maduras
para automatizar o trabalho repetitivo de implementação
algoritmos de aprendizagem baseados em gradiente.
Em :numref:`sec_linear_scratch`, contamos apenas com
(i) tensores para armazenamento de dados e álgebra linear;
e (ii) auto diferenciação para cálculo de gradientes.
Na prática, porque iteradores de dados, funções de perda, otimizadores,
e camadas de rede neural
são tão comuns que as bibliotecas modernas também implementam esses componentes para nós.

Nesta seção, (**mostraremos como implementar
o modelo de regressão linear**) de:numref:`sec_linear_scratch`
(**de forma concisa, usando APIs de alto nível**) de estruturas de *deep learning*.


## Gerando the Dataset

Para começar, vamos gerar o mesmo conjunto de dados como em
:numref:`sec_linear_scratch`.


In [1]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

In [2]:
true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

## Lendo o Dataset

Em vez de usar nosso próprio iterador,
podemos [**chamar a API existente em uma estrutura para ler os dados.**]
Passamos *`features`* e *`labels`* como argumentos e especificamos *`batch_size`*
ao instanciar um objeto iterador de dados.
Além disso, o valor booleano `is_train`
indica se ou não
queremos que o objeto iterador de dados embaralhe os dados
em cada época (passe pelo conjunto de dados).


In [3]:
def load_array(data_arrays, batch_size, is_train=True):  #@save
    """Construct a PyTorch data iterator."""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

In [4]:
batch_size = 10
data_iter = load_array((features, labels), batch_size)

Now we can use `data_iter` in much the same way as we called
the `data_iter` function in :numref:`sec_linear_scratch`.
To verify that it is working, we can read and print
the first minibatch of examples.
Comparing with :numref:`sec_linear_scratch`,
here we use `iter` to construct a Python iterator and use `next` to obtain the first item from the iterator.


In [5]:
next(iter(data_iter))

[tensor([[ 0.4291,  0.1270],
         [ 1.7995, -1.2012],
         [ 0.9239, -0.7505],
         [ 1.3561, -0.4303],
         [ 0.6144, -0.5138],
         [-1.0876, -0.8626],
         [ 1.1090,  3.4219],
         [-1.6905,  0.1326],
         [ 0.6009,  0.9365],
         [ 0.1519, -1.1885]]),
 tensor([[ 4.6351],
         [11.8713],
         [ 8.5979],
         [ 8.3702],
         [ 7.1819],
         [ 4.9432],
         [-5.2131],
         [ 0.3592],
         [ 2.2248],
         [ 8.5418]])]

## Definindo o Modelo


Quando implementamos a regressão linear do zero
em :numref:`sec_linear_scratch`,
definimos nossos parâmetros de modelo explicitamente
e codificamos os cálculos para produzir saída
usando operações básicas de álgebra linear.
Você *deveria* saber como fazer isso.
Mas quando seus modelos ficam mais complexos,
e uma vez que você tem que fazer isso quase todos os dias,
você ficará feliz com a ajuda.
A situação é semelhante a codificar seu próprio blog do zero.
Fazer uma ou duas vezes é gratificante e instrutivo,
mas você seria um péssimo desenvolvedor da web
se toda vez que você precisava de um blog você passava um mês
reinventando tudo.

Para operações padrão, podemos [**usar as camadas predefinidas de uma estrutura,**]
o que nos permite focar especialmente
nas camadas usadas para construir o modelo
em vez de ter que se concentrar na implementação.
Vamos primeiro definir uma variável de modelo `net`,
que se refere a uma instância da classe `Sequential`.
A classe `Sequential` define um contêiner
para várias camadas que serão encadeadas.
Dados dados de entrada, uma instância `Sequential` passa por
a primeira camada, por sua vez passando a saída
como entrada da segunda camada e assim por diante.
No exemplo a seguir, nosso modelo consiste em apenas uma camada,
portanto, não precisamos realmente de `Sequencial`.
Mas como quase todos os nossos modelos futuros
envolverão várias camadas,
vamos usá-lo de qualquer maneira apenas para familiarizá-lo
com o fluxo de trabalho mais padrão.

Lembre-se da arquitetura de uma rede de camada única, conforme mostrado em :numref:`fig_single_neuron`.
Diz-se que a camada está *totalmente conectada*
porque cada uma de suas entradas está conectada a cada uma de suas saídas
por meio de uma multiplicação de matriz-vetor.


: begin_tab: `pytorch`
No PyTorch, a camada totalmente conectada é definida na classe `Linear`. Observe que passamos dois argumentos para `nn.Linear`. O primeiro especifica a dimensão do recurso de entrada, que é 2, e o segundo é a dimensão do recurso de saída, que é um escalar único e, portanto, 1.


In [6]:
# `nn` is an abbreviation for neural networks
from torch import nn

net = nn.Sequential(nn.Linear(2, 1))

## Inicializando os Parâmetros do Modelo


Antes de usar `net`, precisamos (**inicializar os parâmetros do modelo,**)
como os pesos e *bias* no modelo de regressão linear.
As estruturas de *deep learning* geralmente têm uma maneira predefinida de inicializar os parâmetros.
Aqui especificamos que cada parâmetro de peso
deve ser amostrado aleatoriamente a partir de uma distribuição normal
com média 0 e desvio padrão 0,01.
O parâmetro bias será inicializado em zero.


As we have specified the input and output dimensions when constructing `nn.Linear`. Now we access the parameters directly to specify their initial values. We first locate the layer by `net[0]`, which is the first layer in the network, and then use the `weight.data` and `bias.data` methods to access the parameters. Next we use the replace methods `normal_` and `fill_` to overwrite parameter values.


In [7]:
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

## Definindo a Função de Perda


[**A classe `MSELoss` calcula o erro quadrático médio, também conhecido como norma $ L_2 $ quadrada.**]
Por padrão, ela retorna a perda média sobre os exemplos.


In [8]:
loss = nn.MSELoss()

## Definindo o Algoritmo de Otimização


O gradiente descendente estocástico de *minibatch* é uma ferramenta padrão
para otimizar redes neurais
e, portanto, PyTorch o suporta ao lado de uma série de
variações deste algoritmo no módulo `optim`.
Quando nós (**instanciamos uma instância `SGD`,**)
iremos especificar os parâmetros para otimizar
(podem ser obtidos de nossa rede via `net.parameters ()`), com um dicionário de hiperparâmetros
exigido por nosso algoritmo de otimização.
O gradiente descendente estocástico de *minibatch* requer apenas que definamos o valor `lr`, que é definido como 0,03 aqui.


In [9]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

## Treinamento


Você deve ter notado que expressar nosso modelo por meio
APIs de alto nível de uma estrutura de *deep learning*
requer comparativamente poucas linhas de código.
Não tivemos que alocar parâmetros individualmente,
definir nossa função de perda ou implementar o gradiente descendente estocástico de *minibatch*.
Assim que começarmos a trabalhar com modelos muito mais complexos,
as vantagens das APIs de alto nível aumentarão consideravelmente.
No entanto, uma vez que temos todas as peças básicas no lugar,
[**o loop de treinamento em si é surpreendentemente semelhante
ao que fizemos ao implementar tudo do zero.**]

Para refrescar sua memória: para anguns números de épocas,
faremos uma passagem completa sobre o conjunto de dados (*`train_data`*),
pegando iterativamente um *minibatch* de entradas
e os *labels* de verdade fundamental correspondentes.
Para cada *minibatch*, passamos pelo seguinte ritual:

* Gerar previsões chamando `net (X)` e calcular a perda `l` (a propagação direta).
* Calcular gradientes executando a retropropagação.
* Atualizar os parâmetros do modelo invocando nosso otimizador.

Para uma boa medida, calculamos a perda após cada época e a imprimimos para monitorar o progresso.


In [10]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X) ,y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000239
epoch 2, loss 0.000098
epoch 3, loss 0.000099


Abaixo, nós [**comparamos os parâmetros do modelo aprendidos pelo treinamento em dados finitos
e os parâmetros reais**] que geraram nosso *dataset*.
Para acessar os parâmetros,
primeiro acessamos a camada que precisamos de `net`
e, em seguida, acessamos os pesos e a polarização dessa camada.
Como em nossa implementação do zero,
observe que nossos parâmetros estimados são
perto de suas contrapartes verdadeiras.


In [11]:
w = net[0].weight.data
print('error in estimating w:', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('error in estimating b:', true_b - b)

error in estimating w: tensor([0.0004, 0.0001])
error in estimating b: tensor([-0.0005])


## Resumo


* Usando as APIs de alto nível do PyTorch, podemos implementar modelos de forma muito mais concisa.
* No PyTorch, o módulo `data` fornece ferramentas para processamento de dados, o módulo` nn` define um grande número de camadas de rede neural e funções de perda comuns.
* Podemos inicializar os parâmetros substituindo seus valores por métodos que terminam com `_`.


## Exercícios


1. Se substituirmos `nn.MSELoss (*reduction* = 'sum')` por `nn.MSELoss ()`, como podemos alterar a taxa de aprendizagem para que o código se comporte de forma idêntica. Por quê?
1. Revise a documentação do PyTorch para ver quais funções de perda e métodos de inicialização são fornecidos. Substitua a perda pela perda de Huber.
1. Como você acessa o gradiente de `net[0].weight`?

[Discussions](https://discuss.d2l.ai/t/45)


<!--stackedit_data:
eyJoaXN0b3J5IjpbLTUxMjc1MTc4MiwxMzgxNzE4MjMxLDE1MT
Y2Nzk5ODgsLTMwMDU2OTMzNSwtNTUxNzY3NTUsMzYzNjY2MzMs
LTY1Mjk5MTk1OCwtMjE0NTk5NTMwN119
-->
